# Read the Data

In [23]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql._
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType,DoubleType};
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql._
import org.apache.spark.ml._
import org.apache.spark.ml.feature._


In [10]:
val conf = new SparkConf().setAppName("Risk Analysys")
val sc = new SparkContext(conf)
val spark = new SQLContext(sc) 

conf = org.apache.spark.SparkConf@55502758
sc = org.apache.spark.SparkContext@fa7da4e
spark = org.apache.spark.sql.SQLContext@38a22951


org.apache.spark.sql.SQLContext@38a22951

In [11]:
val schema = StructType(Seq(StructField("age",IntegerType,true), StructField("ed",IntegerType,true), 
                        StructField("employ",IntegerType,true), StructField("address",IntegerType,true), 
                        StructField("income",DoubleType,true), StructField("debtinc",DoubleType,true), 
                        StructField("creddebt",DoubleType,true), 
                        StructField("othdebt",DoubleType,true), 
                        StructField("default",DoubleType,true)))

schema = StructType(StructField(age,IntegerType,true), StructField(ed,IntegerType,true), StructField(employ,IntegerType,true), StructField(address,IntegerType,true), StructField(income,DoubleType,true), StructField(debtinc,DoubleType,true), StructField(creddebt,DoubleType,true), StructField(othdebt,DoubleType,true), StructField(default,DoubleType,true))


StructType(StructField(age,IntegerType,true), StructField(ed,IntegerType,true), StructField(employ,IntegerType,true), StructField(address,IntegerType,true), StructField(income,DoubleType,true), StructField(debtinc,DoubleType,true), StructField(creddebt,DoubleType,true), StructField(othdebt,DoubleType,true), StructField(default,DoubleType,true))

In [13]:
val bank_data = spark.read.format("csv")
              .option("header","true")
              .option("delimiter", ",")
             .schema(schema)
            .load("/user/viswatejaster9073/ML/bankloans.csv")

bank_data = [age: int, ed: int ... 7 more fields]


[age: int, ed: int ... 7 more fields]

In [23]:
bank_data.show()

+----+---+------+-------+------+-------+---------+---------+-------+
| age| ed|employ|address|income|debtinc| creddebt|  othdebt|default|
+----+---+------+-------+------+-------+---------+---------+-------+
|  41|  3|    17|     12|  null|    9.3|11.359392| 5.008608|    1.0|
|  27|  1|    10|      6|  31.0|   17.3| 1.362202| 4.000798|    0.0|
|  40|  1|    15|     14|  55.0|    5.5| 0.856075| 2.168925|    0.0|
|  41|  1|    15|     14| 120.0|    2.9|  2.65872|  0.82128|    0.0|
|  24|  2|     2|      0|  28.0|   17.3| 1.787436| 3.056564|    1.0|
|  41|  2|     5|      5|  25.0|   10.2|   0.3927|   2.1573|    0.0|
|  39|  1|    20|      9|  67.0|   30.6| 3.833874|16.668126|    0.0|
|null|  1|    12|     11|  38.0|    3.6| 0.128592| 1.239408|    0.0|
|  24|  1|     3|      4|  19.0|   24.4| 1.358348| 3.277652|    1.0|
|  36|  1|     0|     13|  25.0|   19.7|   2.7777|   2.1473|    0.0|
|  27|  1|     0|      1|  16.0|    1.7| 0.182512| 0.089488|    0.0|
|  25|  1|     4|      0|  23.0|  

## 1) Count of the records 2) Data types

In [14]:
bank_data.count()

850

In [15]:
bank_data.schema

StructType(StructField(age,IntegerType,true), StructField(ed,IntegerType,true), StructField(employ,IntegerType,true), StructField(address,IntegerType,true), StructField(income,DoubleType,true), StructField(debtinc,DoubleType,true), StructField(creddebt,DoubleType,true), StructField(othdebt,DoubleType,true), StructField(default,DoubleType,true))

### writing the stats to a single CSV file, I am using repartition value as 1 because I want to write all the data in single file

In [16]:
val desc = bank_data.describe()

desc = [summary: string, age: string ... 8 more fields]


[summary: string, age: string ... 8 more fields]

In [18]:
desc.repartition(1).write.format("csv").option("header", "true").save("/user/viswatejaster9073/ML/stats.csv")

lastException: Throwable = null


# Finding missing values
- as per the below result we have 152 total missing values
- you can also see the result of describe() function to see the number of missing values

In [31]:
bank_data.map(x=>x.anyNull).show(5)

+-----+
|value|
+-----+
| true|
|false|
|false|
|false|
|false|
+-----+
only showing top 5 rows



In [32]:
bank_data.map(x=>x.anyNull).filter($"value" === true).count

152

# How to fill missing values using Imputer from MLLib
- We know there are some missing values for the coulms default and Income
- If you see below code we are settign Input columns and the output columns once the missing values are filled
- We are filling the missing values with the Mean

In [24]:
val imputer = new Imputer().setInputCols(Array("default", "income")).setOutputCols(Array("default_I", "income_I"))
                .setStrategy("mean")

imputer = imputer_17ff863f84d1


imputer_17ff863f84d1

In [25]:
val model = imputer.fit(bank_data)
val model_final=model.transform(bank_data)

model = imputer_17ff863f84d1
model_final = [age: int, ed: int ... 9 more fields]


[age: int, ed: int ... 9 more fields]

### Removing the coulums which has missign values as we already Imputed those with differenet columns names

In [26]:
val final_Bank_Result1 = model_final.drop("default","income")

final_Bank_Result1 = [age: int, ed: int ... 7 more fields]


[age: int, ed: int ... 7 more fields]

In [27]:
val final_Bank_Result = final_Bank_Result1.na.drop

final_Bank_Result = [age: int, ed: int ... 7 more fields]


[age: int, ed: int ... 7 more fields]

In [28]:
final_Bank_Result.printSchema
val fdesc=final_Bank_Result.describe()

root
 |-- age: integer (nullable = true)
 |-- ed: integer (nullable = true)
 |-- employ: integer (nullable = true)
 |-- address: integer (nullable = true)
 |-- debtinc: double (nullable = true)
 |-- creddebt: double (nullable = true)
 |-- othdebt: double (nullable = true)
 |-- default_I: double (nullable = true)
 |-- income_I: double (nullable = true)



fdesc = [summary: string, age: string ... 8 more fields]


[summary: string, age: string ... 8 more fields]

In [ ]:
fdesc.repartition(1).write.format("csv").option("header", "true").save("/user/viswatejaster9073/ML/mydataf.csv")

# Dealing with Outliers, go thru below sites for more information
- http://waytodatascience.com/range-interquartilerange-percentile/
- http://waytodatascience.com/outliers-treatment/

## Handling Outliers, Below are the parameters need to be passed
- col - the name of the numerical column
- probabilities - a list of quantile probabilities Each number must belong to [0, 1]. For example 0 is the minimum, 0.5 is the median, 1 is the maximum.
- relativeError - The relative target precision to achieve (>= 0). If set to zero, the exact quantiles are computed, which could be very expensive. Note that values greater than 1 are accepted but give the same result as 1.

### with the help of below query we found the Q1 and Q2 now lets calculate IQR

In [29]:
val quantiles=final_Bank_Result.stat.approxQuantile("income_I", Array(0.01,0.99),0.0)

quantiles = Array(14.0, 221.0)


[14.0, 221.0]

In [30]:
val IQR = quantiles(1)-quantiles(0)

IQR = 207.0


207.0

#### Lets find the lower and higher outliers

In [31]:
val low_outlier = quantiles(0)-1.5*IQR

low_outlier = -296.5


-296.5

In [32]:
val high_outlier = quantiles(1)+1.5*IQR

high_outlier = 531.5


531.5

### If you see the summary of income we dont have any values less than or equal to -296.5 or greater than or equal to 531.5, so there are no outliers 

In [33]:
final_Bank_Result.select($"income_I").describe().show()

+-------+-----------------+
|summary|         income_I|
+-------+-----------------+
|  count|              849|
|   mean|46.53300702967948|
| stddev|38.30779447345557|
|    min|             13.0|
|    max|            446.0|
+-------+-----------------+



# Finding Correlation matrix

## converting each row in to vector as it is manditory to convert to this format

In [17]:
final_Bank_Result.columns

[age, ed, employ, address, debtinc, creddebt, othdebt, default_I, income_I]

In [18]:
val assembler = new VectorAssembler()
  .setInputCols(final_Bank_Result.columns)
  .setOutputCol("features")

assembler = vecAssembler_4248ef9ea32d


vecAssembler_4248ef9ea32d

In [19]:
val output = assembler.transform(final_Bank_Result.na.drop)

output = [age: int, ed: int ... 8 more fields]


[age: int, ed: int ... 8 more fields]

In [20]:
output.select($"features").show()

+--------------------+
|            features|
+--------------------+
|[41.0,3.0,17.0,12...|
|[27.0,1.0,10.0,6....|
|[40.0,1.0,15.0,14...|
|[41.0,1.0,15.0,14...|
|[24.0,2.0,2.0,0.0...|
|[41.0,2.0,5.0,5.0...|
|[39.0,1.0,20.0,9....|
|[24.0,1.0,3.0,4.0...|
|[36.0,1.0,0.0,13....|
|[27.0,1.0,0.0,1.0...|
|[25.0,1.0,4.0,0.0...|
|[52.0,1.0,24.0,14...|
|[37.0,1.0,6.0,9.0...|
|[48.0,1.0,22.0,15...|
|[36.0,2.0,9.0,6.0...|
|[36.0,2.0,13.0,6....|
|[43.0,1.0,23.0,19...|
|[39.0,1.0,6.0,9.0...|
|[41.0,3.0,0.0,21....|
|[39.0,1.0,22.0,3....|
+--------------------+
only showing top 20 rows



## Finding corelation and storing it in coeff1 variable

In [17]:
val Row(coeff1: Matrix) = Correlation.corr(output,"features").head

coeff1 = 


1.0                   0.013890935310364066  0.5540539797392409    ... (9 total)
0.013890935310364066  1.0                   -0.15073439393182878  ...
0.5540539797392409    -0.15073439393182878  1.0                   ...
0.5999027439746645    0.053294449596498324  0.3445179656366462    ...
0.009393894758528618  0.007176600305510925  -0.03306433740271827  ...
0.2799424858461383    0.09852572619495747   0.38231075953384924   ...
0.3387286921850727    0.14052088251701092   0.4148886090917719    ...
-0.12355818984485614  0.1035707819878161    -0.25193665517801883  ...
0.4770041273132644    0.2140258655626239    0.6245729481921632    ...

### Converting Matrix in to a List of Array to convert it to RDD
- We are doing it because, it is difficult to understand the correlation matrix as you seen above

In [18]:
val localMatrix: List[Array[Double]] = coeff1
    .toArray
    .grouped(coeff1.numCols)
    .toList

localMatrix = List(Array(1.0, 0.013890935310364066, 0.5540539797392409, 0.5999027439746645, 0.009393894758528618, 0.2799424858461383, 0.3387286921850727, -0.12355818984485614, 0.4770041273132644), Array(0.013890935310364066, 1.0, -0.15073439393182878, 0.053294449596498324, 0.007176600305510925, 0.09852572619495747, 0.14052088251701092, 0.1035707819878161, 0.2140258655626239), Array(0.5540539797392409, -0.15073439393182878, 1.0, 0.3445179656366462, -0.03306433740271827, 0.38231075953384924, 0.4148886090917719, -0.25193665517801883, 0.6245729481921632), Array(0.5999027439746645, 0.053294449596498324, 0.3445179656366462, 1.0, -0.03252035769911093, 0.16200501333681244, 0.1857719812004867, -0.1474556689754082, 0.30844726151737173), Array(0.009393894758528618, 0.007176600...


List([D@506a87f9, [D@e405445, [D@389021a, [D@63a27bdf, [D@67f37d1d, [D@6b9dffda, [D@20cfb961, [D@50435224, [D@2c24d6a4)

In [19]:
val cordd=sc.parallelize(localMatrix).repartition(1)
cordd.take(5)

cordd = MapPartitionsRDD[33] at repartition at <console>:85


[[1.0, 0.013890935310364066, 0.5540539797392409, 0.5999027439746645, 0.009393894758528618, 0.2799424858461383, 0.3387286921850727, -0.12355818984485614, 0.4770041273132644], [0.013890935310364066, 1.0, -0.15073439393182878, 0.053294449596498324, 0.007176600305510925, 0.09852572619495747, 0.14052088251701092, 0.1035707819878161, 0.2140258655626239], [0.5540539797392409, -0.15073439393182878, 1.0, 0.3445179656366462, -0.03306433740271827, 0.38231075953384924, 0.4148886090917719, -0.25193665517801883, 0.6245729481921632], [0.5999027439746645, 0.053294449596498324, 0.3445179656366462, 1.0, -0.03252035769911093, 0.16200501333681244, 0.1857719812004867, -0.1474556689754082, 0.30844726151737173], [0.009393894758528618, 0.007176600305510925, -0.03306433740271827, -0.03252035769911093, 1.0, 0.5146104007505761, 0.5723450152301102, 0.35869677227123453, -0.035584612690822116]]

### Converting to Data Frame

In [20]:
val s = cordd.toDF

s = [value: array<double>]


[value: array<double>]

In [24]:
s.show(3)

+--------------------+
|               value|
+--------------------+
|[1.0, 0.013890935...|
|[0.01389093531036...|
|[0.55405397973924...|
+--------------------+
only showing top 3 rows



### If you see above response we have the single column with the name value has list of Array, So we need to split accordingly

In [21]:
s.createOrReplaceTempView("cor")

In [22]:
val corelationmatrix=spark.sql("select value[0] as age,value[1] as ed,value[2] as employ,value[3] as address,value[4] as income,value[5] as debtinc,value[6] as creddebt,value[7] as othdebt,value[8] as default from cor")

corelationmatrix = [age: double, ed: double ... 7 more fields]


[age: double, ed: double ... 7 more fields]

In [23]:
corelationmatrix.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+
|                 age|                  ed|              employ|             address|              income|            debtinc|           creddebt|             othdebt|             default|
+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+
|                 1.0|0.013890935310364066|  0.5540539797392409|  0.5999027439746645|0.009393894758528618| 0.2799424858461383| 0.3387286921850727|-0.12355818984485614|  0.4770041273132644|
|0.013890935310364066|                 1.0|-0.15073439393182878|0.053294449596498324|0.007176600305510925|0.09852572619495747|0.14052088251701092|  0.1035707819878161|  0.2140258655626239|
|  0.5540539797392409|-0.15073439393182878|            

#### Writing correlation matrix in to a file for better view

In [29]:
corelationmatrix.repartition(1).write.format("csv").option("header","true").option("delimiter", ",").save("/user/viswatejaster9073/ML/corelationmatrix")